In [2]:
import datetime

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
test = pd.read_excel('/content/drive/My Drive/flight fare prediction_/Test_set.xlsx')

In [5]:
test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Date_of_Journey  2671 non-null   object
 2   Source           2671 non-null   object
 3   Destination      2671 non-null   object
 4   Route            2671 non-null   object
 5   Dep_Time         2671 non-null   object
 6   Arrival_Time     2671 non-null   object
 7   Duration         2671 non-null   object
 8   Total_Stops      2671 non-null   object
 9   Additional_Info  2671 non-null   object
dtypes: object(10)
memory usage: 208.8+ KB


In [7]:
test.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [8]:
Airline = pd.get_dummies(test['Airline'], drop_first=True)
Source = pd.get_dummies(test['Source'], drop_first=True)
Destination = pd.get_dummies(test['Destination'], drop_first=True)

Grabbing Date and Month of Journey


In [9]:
test['Date_of_Journey'] = pd.to_datetime(test['Date_of_Journey'],format= "%d/%m/%Y")

In [10]:

test['Day_of_journey'] = test['Date_of_Journey'].dt.day_name()
test['Month_of_journey'] = test['Date_of_Journey'].dt.month_name()

test.drop(['Date_of_Journey'], axis = 1 , inplace = True)

In [11]:

test.drop(['Additional_Info', 'Route'], axis=1, inplace=True)

In [12]:
test.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Day_of_journey,Month_of_journey
0,Jet Airways,Delhi,Cochin,17:30,04:25 07 Jun,10h 55m,1 stop,Thursday,June
1,IndiGo,Kolkata,Banglore,06:20,10:20,4h,1 stop,Sunday,May
2,Jet Airways,Delhi,Cochin,19:15,19:00 22 May,23h 45m,1 stop,Tuesday,May
3,Multiple carriers,Delhi,Cochin,08:00,21:00,13h,1 stop,Tuesday,May
4,Air Asia,Banglore,Delhi,23:55,02:45 25 Jun,2h 50m,non-stop,Monday,June


In [13]:
test['Departure_hour'] = pd.to_datetime(test['Dep_Time']).dt.hour
test['Departure_min'] = pd.to_datetime(test['Dep_Time']).dt.minute

test['Arrival_hour'] = pd.to_datetime(test['Arrival_Time']).dt.hour
test['Arrival_min'] = pd.to_datetime(test['Arrival_Time']).dt.minute

test.drop(['Dep_Time', 'Arrival_Time'], axis=1, inplace=True)

In [14]:
test['Total_Stops'].value_counts()

1 stop      1431
non-stop     849
2 stops      379
3 stops       11
4 stops        1
Name: Total_Stops, dtype: int64

In [15]:
test.replace({'non-stop':0, '1 stop':1, '2 stops':2, '3 stops':3, '4 stops':4}, inplace=True)
test.drop(['Airline','Source','Destination'], axis=1, inplace=True)

In [17]:

journey_day = pd.get_dummies(test['Day_of_journey'], drop_first = True)
journey_month = pd.get_dummies(test['Month_of_journey'], drop_first = True)

In [18]:
int(test['Duration'][2].split()[0].split('h')[0])

23

In [19]:
duration = list(test['Duration'])
for i in range(len(duration)):
  if len(duration[i].split()) != 2:
    if'h' in duration[i]:
      duration[i]=duration[i].strip() + ' 0m'
    else:
      duration[i]='0h ' + duration[i]

In [20]:
int(duration[0].split()[1].split('m')[0])

55

In [21]:
duration_hour=[]
duration_min=[]

for i in range(len(duration)):
  duration_hour.append(int(duration[i].split()[0].split('h')[0]))
  duration_min.append(int(duration[i].split()[1].split('m')[0]))

In [22]:
duration_hour=np.array(duration_hour)
duration_min=np.array(duration_min)

duration_hour=pd.DataFrame(duration_hour)
duration_min=pd.DataFrame(duration_min)

duration_hour.rename(columns={0:'duration_hour'}, inplace=True)
duration_min.rename(columns={0:'duration_min'}, inplace=True)

In [23]:
test_final = pd.concat([test, Airline, Source, Destination, duration_hour, duration_min, journey_day, journey_month], axis=1)
test_final.drop('Duration', axis=1, inplace=True)
test_final.head()

,Total_Stops,Day_of_journey,Month_of_journey,Departure_hour,Departure_min,Arrival_hour,Arrival_min,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,Vistara,Vistara Premium economy,Chennai,Delhi,Kolkata,Mumbai,Cochin,Delhi,Hyderabad,Kolkata,New Delhi,duration_hour,duration_min,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,June,March,May
0,1,Thursday,June,17,30,4,25,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,10,55,0,0,0,1,0,0,1,0,0
1,1,Sunday,May,6,20,10,20,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4,0,0,0,1,0,0,0,0,0,1
2,1,Tuesday,May,19,15,19,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,23,45,0,0,0,0,1,0,0,0,1
3,1,Tuesday,May,8,0,21,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,13,0,0,0,0,0,1,0,0,0,1
4,0,Monday,June,23,55,2,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,50,1,0,0,0,0,0,1,0,0


In [25]:
test_final.drop(['Day_of_journey', 'Month_of_journey'], axis=1, inplace=True)

In [26]:
test_final

,Total_Stops,Departure_hour,Departure_min,Arrival_hour,Arrival_min,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,Vistara,Vistara Premium economy,Chennai,Delhi,Kolkata,Mumbai,Cochin,Delhi,Hyderabad,Kolkata,New Delhi,duration_hour,duration_min,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,June,March,May
0,1,17,30,4,25,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,10,55,0,0,0,1,0,0,1,0,0
1,1,6,20,10,20,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4,0,0,0,1,0,0,0,0,0,1
2,1,19,15,19,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,23,45,0,0,0,0,1,0,0,0,1
3,1,8,0,21,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,13,0,0,0,0,0,1,0,0,0,1
4,0,23,55,2,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,50,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,20,30,20,25,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,23,55,0,0,0,1,0,0,1,0,0
2667,0,14,20,16,55,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,35,0,0,0,0,0,1,0,1,0
2668,1,21,50,4,25,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,6,35,0,0,0,0,0,1,0,1,0
2669,1,4,0,19,15,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,15,15,0,0,0,0,0,1,0,1,0


In [27]:
test_final.isnull().sum()

Total_Stops                          0
Departure_hour                       0
Departure_min                        0
Arrival_hour                         0
Arrival_min                          0
Air India                            0
GoAir                                0
IndiGo                               0
Jet Airways                          0
Jet Airways Business                 0
Multiple carriers                    0
Multiple carriers Premium economy    0
SpiceJet                             0
Vistara                              0
Vistara Premium economy              0
Chennai                              0
Delhi                                0
Kolkata                              0
Mumbai                               0
Cochin                               0
Delhi                                0
Hyderabad                            0
Kolkata                              0
New Delhi                            0
duration_hour                        0
duration_min             

In [28]:
test_final.isna().sum()

Total_Stops                          0
Departure_hour                       0
Departure_min                        0
Arrival_hour                         0
Arrival_min                          0
Air India                            0
GoAir                                0
IndiGo                               0
Jet Airways                          0
Jet Airways Business                 0
Multiple carriers                    0
Multiple carriers Premium economy    0
SpiceJet                             0
Vistara                              0
Vistara Premium economy              0
Chennai                              0
Delhi                                0
Kolkata                              0
Mumbai                               0
Cochin                               0
Delhi                                0
Hyderabad                            0
Kolkata                              0
New Delhi                            0
duration_hour                        0
duration_min             

In [29]:
test_final.to_csv('/content/drive/My Drive/flight fare prediction_/Test_final.csv')

Grabbing y_test from Sample Submission.xlsx

In [31]:
y_test = pd.read_excel('/content/drive/My Drive/flight fare prediction_/Sample_submission.xlsx', sheet_name=0)

In [32]:
y_test

,Price
0,15998
1,16612
2,25572
3,25778
4,16934
...,...
2666,14046
2667,14086
2668,22720
2669,23544


In [33]:
test_final.shape

(2671, 35)

In [34]:
y_test.isnull().sum()

Price    0
dtype: int64

In [35]:
y_test.shape

(2671, 1)

In [36]:
y_test.to_csv('/content/drive/My Drive/flight fare prediction_/y_test.csv')

In [37]:
test_final.to_csv('/content/drive/My Drive/flight fare prediction_/x_test.csv')